In [42]:
# show figures inline in the notebook
%matplotlib qt
import matplotlib.pyplot as plt  # Import library for direct plotting functions
import numpy as np               # Import Numerical Python
import scipy.constants as const
from IPython.core.display import display, HTML #Import HTML for formatting output

# NOTE: Uncomment following lines ONLY if you are not using installation via pip
# import sys, os
# rootDir = '/path/to/arc/directory' # e.g. '/Users/Username/Desktop/ARC-Alkali-Rydberg-Calculator'
# sys.path.insert(0,rootDir)

from arc import *                 #Import ARC (Alkali Rydberg Calculator)

In [2]:
#Declare atom variables
cs=Caesium() #Cesium has 55 electrons w/ valence 7S 
rb=Rubidium() #Rubidium has 37 electrons w/ valences 6S
cs_gnd=9
rb_gnd=12
E_TO_COULOMB = .000000000000000000160217620898
A0_TO_METER = .00000000005291772
TOTAL_STATES = 250
POT = 20000000 #20MHz


atom = rb
atom_gnd = rb_gnd
name = "Caesium"
TEMP = 1

In [3]:
#Electron state generator for use with the ARC python package
#Given a Z, this class generates the list of occupied electron states for in input to the ARC energy level calculator

class State:
    
    #Definition of the principal QN, angular QN, magnetic QN, and spin QN
    def __init__(self,n,l,j,mj):
        self.n = n
        self.l = int(l)
        self.m = np.linspace(0,l,num= l+1)
        
        if j:
            self.j = j
        else:
            if self.l == 0:
                self.j = [.5]
            else:
                self.j = np.linspace(np.absolute(l-.5),l+.5,num=2)
        if mj:
            self.mj = mj
        
        else:
            self.mj = [[jj,None] for jj in self.j]

            for jj in self.mj:
                jj[1] = np.linspace(-jj[0],jj[0],num=(2*jj[0]+1))
            
class Transition:
    
    def __init__(self,s1,s2,atom,j):
        if not j:
            j = s2.j
        self.s1 = s1
        self.s2 = s2
        self.wvln = atom.getTransitionWavelength(s1.n,s1.l,s1.j,s2.n,s2.l,j)
        self.time = atom.getStateLifetime(s2.n,s2.l,j,0)
        self.rate = atom.getTransitionRate(s1.n,s1.l,s1.j,s2.n,s2.l,j,0)
        self.dipole = atom.getDipoleMatrixElement(s1.n,s1.l,s1.j,s1.mj,s2.n,s2.l,j,j,1)*E_TO_COULOMB*A0_TO_METER
        
#Electron state generating class
class ElectronStates:

    def aufbau(self,n,l,e_list):

        e_list.append(State(n,l,None,None))
        self.ec -= 2*l+1
        
        #Conditionally recurses or returns remaining electron counts
        if l != 0:
            return self.aufbau(n+1,l-1,e_list)
        else:
            return self.ec
                
    
    #Class constructor for electron state generating class
    def __init__(self,Z,e_list):

        self.occ_n = []
        self.Z = Z
        self.ec = Z
        n=1
        l=0
         
        while self.ec > 0:

            self.aufbau(n,l,e_list)
            
            if (n+l)%2 == 0 and n < 5:
                l+=1
            else:
                n+=1




In [4]:
#TRANSITION INFO

trans_dict={}
pqn = {}


def calcPower(elemSum,wvln):
    NA = .5 #Typical numerical aperture
    return (POT*(const.epsilon_0)*((wvln*const.Planck*const.c/(const.pi*NA))**2)/elemSum)

def calcAbsValueDipoleDetuningSum(trans,wvln):
    return (trans.wvln*(trans.dipole)**2./(1-(wvln/trans.wvln)**2))

def calcDipoleDetuningSum(trans,wvln):
    return (trans.wvln*(trans.dipole)**2./(1-(wvln/trans.wvln)**2))

def createSpectrum(ground,wavelengths,states,atom):
    print(ground)
    power = []
    polar = []
    absElemSum = 0
    elemSum = 0
    for wvln in wavelengths:
        absElemSum = 0
        elemSum = 0
        for st in states:
            if not (ground.n < st.n and ground.l==st.l) and not ground.n > st.n \
            and not (ground.n == st.n and ground.l >= st.l):
                for j in st.j:
                    trans = Transition(ground,st,atom,j) 

                    if trans.dipole != 0.0:
                        print("From: " +str(ground.n)+str(ground.l)+str(ground.j) + " to " +str(st.n)+str(st.l)+str(j))
                        print("Wvln: " + str(trans.wvln))
                        absElemSum += calcAbsValueDipoleDetuningSum(trans,wvln)
                        elemSum += calcDipoleDetuningSum(trans,wvln)
        power.append(calcPower(absElemSum,wvln))
        polar.append(elemSum)
    return power,polar


In [11]:
#MAIN

st = []
ElectronStates(TOTAL_STATES,st)

#Lasers
wavelengths = np.linspace(.0000002,.000001,200)

# +1 Polarization
#CESIUM 6S to 7P transition
cs_t = Transition(State(6,0,1/2,1/2),State(7,1,3/2,3/2),cs,None)
p_cs,polar_cs = createSpectrum(cs_t.s1,wavelengths,st,cs)

# createSpectrum(cs_transition)
#RUBIDIUM 5S to 6P transition

rb_t = Transition(State(5,0,1/2,1/2),State(6,1,3/2,3/2),rb,None)
p_rb,polar_rb = createSpectrum(rb_t.s1,wavelengths,st,rb)

/Users/jordankemp/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
/Users/jordankemp/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.


From: 600.5 to 611.5
Wvln: 8.523473060245015e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 600.5 to 711.5
Wvln: 4.556556792845617e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 600.5 to 811.5
Wvln: 3.877245321293302e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 600.5 to 911.5
Wvln: 3.6124992311015295e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not tri

6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 600.5 to 1211.5
Wvln: 3.348474967347318e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 600.5 to 1311.5
Wvln: 3.3140919917262387e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 600.5 to 1411.5
Wvln: 3.28956434186693e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol

6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 600.5 to 1611.5
Wvln: 3.257626874305657e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 600.5 to 1711.5
Wvln: 3.246873835633732e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 600.5 to 1811.5
Wvln: 3.23832878119711e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol 

6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 600.5 to 811.5
Wvln: 3.877245321293302e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 600.5 to 911.5
Wvln: 3.6124992311015295e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 600.5 to 1011.5
Wvln: 3.477815591420838e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 600.5 to 1111.5
Wvln: 3.398953206086976e-07
6j-Symbol is not triangular!
6j-Symbol is not t

...

6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 500.5 to 1811.5
Wvln: 3.0097368609945634e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 500.5 to 1911.5
Wvln: 3.004748344523267e-07
From: 500.5 to 511.5
Wvln: 7.802414991382089e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 500.5 to 611.5
Wvln: 4.202989214582461e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 500.5 to 711.5
Wvln: 3.588073615714269e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular

From: 500.5 to 911.5
Wvln: 3.2289097047721087e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 500.5 to 1011.5
Wvln: 3.158452518890207e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 500.5 to 1111.5
Wvln: 3.1134764233924656e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 500.5 to 1211.5
Wvln: 3.0829102573189415e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is no

6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 500.5 to 1211.5
Wvln: 3.0829102573189415e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 500.5 to 1311.5
Wvln: 3.0611466230111556e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 500.5 to 1411.5
Wvln: 3.045079982436423e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symb

From: 500.5 to 1411.5
Wvln: 3.045079982436423e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 500.5 to 1511.5
Wvln: 3.032870505135045e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 500.5 to 1611.5
Wvln: 3.0233690268441615e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 500.5 to 1711.5
Wvln: 3.015826439852207e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not

6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 500.5 to 1711.5
Wvln: 3.015826439852207e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 500.5 to 1811.5
Wvln: 3.0097368609945634e-07
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
From: 500.5 to 1911.5
Wvln: 3.004748344523267e-07


In [57]:
wvlns=wavelengths*1000000000
plt.plot(wvlns[:180],np.array(p_cs[:180])*1000)
plt.title('Power density for 20MHz Cesium traps at various wavelengths')
plt.xlabel("Wavelength (nm)")
plt.ylabel("Power Density (mW/atom)")
plt.rcParams['figure.figsize'] = [15, 10]
plt.grid()

plt.axvline(x=852, color="black")
plt.axhline(y=0, color="black")


plt.ylim(top=(1))





print(p_cs)

plt.show()


[0.00017352759595380706, 0.00018013424886630897, 0.00018683674797017967, 0.0001936329338374181, 0.00020052059556347257, 0.00020749746918895766, 0.00021456123583488395, 0.00022170951948487523, 0.00022893988432869707, 0.00023624983155559623, 0.00024363679545071488, 0.0002510981385990667, 0.00025863114593309075, 0.00026623301726207404, 0.00027390085777980025, 0.00028163166583653403, 0.0002894223169432132, 0.0002972695424821328, 0.00030516990081175276, 0.0003131197371599427, 0.0003211151264971349, 0.00032915178967150995, 0.0003372249658092796, 0.0003453292096326576, 0.000353458052029917, 0.000361603392422947, 0.0003697543122229385, 0.000377894465777613, 0.00038599525486353824, 0.0003939920503884857, 0.00040162911520526896, 0.00040023262371879497, 0.0004143807945103713, 0.00042823711861688556, 0.00043690171438794844, 0.0004520764247596576, 0.0004514375931392815, 0.0004742675578980985, 0.00047140900345528104, 0.00047561908242450146, 0.0004353119848645624, 0.0005046831052654486, 0.00050829177

In [58]:
plt.plot(wvlns[50:180],np.array(polar_cs[50:180]))
plt.title('Polarity of Cesium in traps by wavelength')
plt.xlabel("Wavelength (nm)")
plt.ylabel("Polarity (Cm)")
plt.rcParams['figure.figsize'] = [15, 10]
plt.grid()
plt.axhline(y=0, color="black")
plt.ylim(top=(1.5*10**(-61)))

print(polar_cs)



[6.528055878832628e-64, 6.543981206463305e-64, 6.560314381053206e-64, 6.577062144628118e-64, 6.594231574756604e-64, 6.611830113665092e-64, 6.629865602133474e-64, 6.648346319268023e-64, 6.667281029566482e-64, 6.686679039121065e-64, 6.706550263393812e-64, 6.7269053098163e-64, 6.747755579616384e-64, 6.769113394921382e-64, 6.790992159585607e-64, 6.813406565753214e-64, 6.836372863575833e-64, 6.859909219917704e-64, 6.88403620532939e-64, 6.908777470746801e-64, 6.934160713258818e-64, 6.960219097728843e-64, 6.986993427024148e-64, 7.014535602737262e-64, 7.042914446320539e-64, 7.072226170035409e-64, 7.10261492994283e-64, 7.13431826866475e-64, 7.167786695467822e-64, 7.204102925698237e-64, 7.247737209190288e-64, 7.456565193463579e-64, 7.381457899727286e-64, 7.318430280309796e-64, 7.347712005481836e-64, 7.271663898063282e-64, 7.454813427997587e-64, 7.262427204025393e-64, 7.475885146324474e-64, 7.579554776878046e-64, 8.46904681428181e-64, 7.468626547752609e-64, 7.57993194960366e-64, 7.652027018533592

In [59]:
wvlns=wavelengths*1000000000
plt.plot(wvlns,np.array(p_rb)*1000)
plt.title('Power density for 20MHz Rubidium traps at various wavelengths')
plt.xlabel("Wavelength (nm)")
plt.ylabel("Power Density (mW/atom)")
plt.grid()
plt.rcParams['figure.figsize'] = [15, 10]

print(p_rb)

plt.axhline(y=0, color="black")
plt.ylim(top=2,bottom=-2)


[0.00021133618727156938, 0.0002192724126955943, 0.00022731420969595515, 0.00023545838785849027, 0.00024370167596249445, 0.00025204071826462516, 0.00026047206992794406, 0.00026899219133780136, 0.00027759744094747654, 0.00028628406615121947, 0.0002950481914640828, 0.0003038858029522887, 0.0003127927273279129, 0.00032176460325972884, 0.0003307968410020758, 0.0003398845639080659, 0.00034902252075510754, 0.0003582049488623179, 0.00036742534957781786, 0.00037667609678934075, 0.00038594769864215117, 0.00039522725189087985, 0.00040449468966794844, 0.0004137113062058095, 0.00042276542811812187, 0.00043928081803030144, 0.0005046451455188104, 0.00045797667369531836, 0.0004626824148927201, 0.00047684791355906615, 0.000478323386391097, 0.0004954878717443238, 0.0005004245819216302, 0.0005060524467795103, 0.0005253981773584167, 0.0005298293948853734, 0.0005375766436719864, 0.0005452101464806456, 0.0005513743085147824, 0.0005470182158020261, 0.0005980132597421202, 0.0005918731208963989, 0.000597616166

(-2, 2)

In [60]:
plt.plot(wvlns[50:180],np.array(polar_rb[50:180])*1000)
plt.title('Polarity of Rubidium in traps by wavelength')
plt.xlabel("Wavelength (nm)")
plt.ylabel("Polarity (Cm)")
plt.rcParams['figure.figsize'] = [15, 10]

plt.grid()
plt.axhline(y=0, color="black")
plt.ylim(top=(1.5*10**(-58)))
print(polar_rb)

[5.3601697727719725e-64, 5.375939110306486e-64, 5.392130154368407e-64, 5.408751205163073e-64, 5.425811034870142e-64, 5.443318939650084e-64, 5.4612848026525963e-64, 5.479719171337936e-64, 5.498633353701103e-64, 5.518039539875239e-64, 5.537950958432916e-64, 5.5583820810880345e-64, 5.5793488964436555e-64, 5.600869284755768e-64, 5.6229635447900895e-64, 5.645655157360086e-64, 5.6689719316196075e-64, 5.692947799159853e-64, 5.7176257663733984e-64, 5.743063083046125e-64, 5.7693410335738796e-64, 5.796585537972456e-64, 5.825017434646882e-64, 5.8551071708646015e-64, 5.888312182694089e-64, 5.821654099384656e-64, 5.2040974172166976e-64, 5.8868050398089315e-64, 5.979764009328276e-64, 5.9523365882442005e-64, 6.085636549218036e-64, 6.023075077103258e-64, 6.112278452405976e-64, 6.193080412876541e-64, 6.110085855577622e-64, 6.204540270555447e-64, 6.260285060470533e-64, 6.31744224992722e-64, 6.391664450721273e-64, 6.5902391986699e-64, 6.1648759765389e-64, 6.368408202891727e-64, 6.446975973969958e-64, 6.5

In [56]:
%matplotlib qt

wvlns=wavelengths*1000000000
plt.plot(wvlns,np.array(p_rb)*1000)
plt.plot(wvlns,np.array(p_cs)*1000)

plt.axhline(y=0,color="black")



plt.title('Power density for 20MHz Rubidium and Cesium traps at various wavelengths')
plt.xlabel("Wavelength (nm)")
plt.ylabel("Power Density (mW/atom)")
plt.rcParams['figure.figsize'] = [15, 10]
plt.grid()
plt.ylim(top=2,bottom=-3)




(-3, 2)

In [13]:
compare to actual results

SyntaxError: invalid syntax (<ipython-input-13-4d7dfc66a561>, line 1)

In [ ]:
from arc import *
atom = Rubidium()
# transition 5 S_{1/2} m_j=-0.5 -> 5 P_{3/2} m_j=-1.5 for laser
# driving sigma- transition
print(atom.getDipoleMatrixElement(5,0,0.5,-0.5,5,1,1.5,-1.5,-1))

In [ ]:
[938]